## Projektissa murteiden luokittelua on kokeiltu alla olevalla scikit-learn-classifierilla
- Luokitteluun sopivat txt-tiedostot löytyvät Altaasta, kunkin paikkakunnan korpus on varastoitu sellaisenaan yhdistelemättä muihin.
- Tiedostoissa kunkin rivin muoto on: LABEL tab TEXT
- Aiemmissa testeissä EP (Etelä-Pohjanmaa), PI (Pirkanmaa), SA (Satakunta),  VS (Varsinais-Suomi), KH (Kanta-Häme) ovat luokittelua varten jäsennelty LAN-labelilla.
- Vastaavasti KA (Kainuu), ES (Etelä-Savo), PS (Pohjois-Savo), PK (Pohjois-Karjala), KS (Keski-Suomi) ovat jäsennelty ITA-labelilla.
- Lisäksi paikkakunnat KP (Keski-Pohjanmaa), KY (Kymenlaakso), PH (Päijät-Häme), AH (Ahvenanmaa), EK (Etelä-Karjala), IU (Itä-Uusimaa), LA (Lappi), PO (Pohjanmaa), UU (Uusimaa).
- Luokittelua on testattu myös murrepiirteistä johdetuilla merkkijonoilla, tässä tiedostossa liitettynä skripti muokkaamaan tiedosto edellä mainittuun muotoon (substringitjaljella.py).
 - Huom. tällaisenaan tarkastelussa vain kommentit, eli ei mahdollista huomioida annotaatioita. Seurauksena mukana on paljon roskaa.
 - Hakua on pyritty tarkentamaan luomalla training ja testing datat riveiltä, joista murrepiirteitä on löytynyt. Alustavasti vastaavaan seulontaan toimii Unix-komennotkin, mutta järkevää olisi luoda aineisto niin, että annotaatioiden avulla riveille saataisiin vain todelliset murrepiirteet.  


## sklearnsvm.py

In [ ]:
import sklearn.feature_extraction
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score, recall_score, f1_score
import sys
from sklearn.metrics import classification_report

labels_train=[]
labels_test=[]

feats_train=[]
feats_test=[]

def read_file(file):
    f_op = open(file,"r")
    labels = []
    feats = []
    for line in f_op:
        line=line.strip().split("\t")
        if len(line) == 1:
            continue
        else:
            feats.append(line[1])
            labels.append(line[0])
    return(labels,feats)

labels_train,feats_train = read_file(sys.argv[1])
labels_test,feats_test = read_file(sys.argv[2])

def data_iterator(f):
    for token in f:
        yield token


def tokenizer(txt):
    """Simple whitespace tokenizer"""
    return txt.split()


iterator=data_iterator(feats_train)
test_iterator=data_iterator(feats_test)
vectorizer=sklearn.feature_extraction.text.TfidfVectorizer(tokenizer=tokenizer,use_idf=True,min_df=0.0001)
d=vectorizer.fit_transform(iterator)

d_test=vectorizer.transform(test_iterator)

classifier=SVC(C=0.7,kernel='linear',shrinking=False)
classifier.fit(d,labels_train)
labels_test_pred = classifier.predict(d_test)
print(classification_report(labels_test, labels_test_pred))

## substringitjaljella.py

In [ ]:
import sys

def process_feature(feature):
    linguistic_features = [
        ('elä', 3),
        ('tt', 2),
        ('ts', 2),
        ('eir', 3),
        ('yä', 2),
        ('ht', 2),
        ('eä', 2),
        ('eiä', 3),
        ('ua', 2),
        ('ae',2),
        ('hr',2)
    ]
    result = ''
    i = 0
    while i < len(feature):
        found = False
        for linguistic_feature, length in linguistic_features:
            if i + len(linguistic_feature) <= len(feature) and feature[i:i + len(linguistic_feature)].lower() == linguistic_feature:
                result += linguistic_feature
                i += length
                found = True
                break
        if not found:
            result += ' '
            i += 1
    return result

for line in sys.stdin:
    if line.strip() != '':
        label, feature = line.split('\t', 1)
        processed_feature = process_feature(feature)
        print(f"{label}\t{processed_feature}")